# Ejercicio 6: Dense Retrieval e Introducción a FAISS

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [ ]:
corpus_limitado = newsgroupsdocs[:2000]

## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

model_sbert = SentenceTransformer('all-MiniLM-L6-v2')
model_e5 = SentenceTransformer('intfloat/e5-base')

print(f"Modelos cargados: {'all-MiniLM-L6-v2'} y {'intfloat/e5-base'}")
print(f"El corpus tiene {len(corpus_limitado)} documentos.")

SBERT

In [ ]:
# Generar los embeddings
embeddings_sbert = model_sbert.encode(
    corpus_limitado,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings_sbert_np = embeddings_sbert

E5

In [ ]:
# Anteponer "passage: " a cada documento
corpus_e5 = [f"passage: {doc}" for doc in corpus_limitado]

# Generar los embeddings con el corpus modificado
embeddings_e5 = model_e5.encode(
    corpus_e5,
    show_progress_bar=True,
    convert_to_numpy=True
)
# El resultado ya es un array de NumPy
embeddings_e5_np = embeddings_e5


## Parte 3: Consulta
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con similitud coseno.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k(query_embedding, doc_embeddings, corpus, k=5):

    similarity_scores = cosine_similarity(query_embedding, doc_embeddings)[0]

    top_indices = np.argsort(similarity_scores)[::-1][:k]

    results = []
    for i in top_indices:
        results.append({
            'index': i,
            'score': similarity_scores[i],
            'text': corpus[i]
        })
    return results

QUERY = "space exploration"

# SBERT
query_embedding_sbert = model_sbert.encode(QUERY, convert_to_numpy=True).reshape(1, -1)

results_sbert = retrieve_top_k(query_embedding_sbert, embeddings_sbert_np, corpus_limitado, k=5)

for rank, res in enumerate(results_sbert):
    print(f"\n[{rank+1}. Documento #{res['index']}] (Similitud: {res['score']:.4f})")
    print("-" * 20)
    print(res['text'][:500].strip() + "...")


# Búsqueda con E5 ('intfloat/e5-base')

print("\n\n" + "="*60)
print("  RESULTADOS CON E5 ('intfloat/e5-base') (Con prefijo 'query: ')")
print("="*60)

QUERY_E5_PREFIXED = f"query: {QUERY}"
query_embedding_e5 = model_e5.encode(QUERY_E5_PREFIXED, convert_to_numpy=True).reshape(1, -1)

results_e5 = retrieve_top_k(query_embedding_e5, embeddings_e5_np, corpus_limitado, k=5)

for rank, res in enumerate(results_e5):
    print(f"\n[{rank+1}. Documento #{res['index']}] (Similitud: {res['score']:.4f})")
    print("-" * 20)
    print(res['text'][:500].strip() + "...")